In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
## csv of s&p 500 data from yahoo finance
filepath = "../Input/^GSPC.csv"

# Load spreadsheet
xl = pd.ExcelFile("../Input/government_shutdown_stats.xlsx")

# Load a sheet into a DataFrame by name:
shutdown_df = xl.parse(xl.sheet_names[0])


In [5]:
stock_prices_unclean = pd.read_csv(filepath)

In [7]:
stock_prices_unclean.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1977-01-10,105.010002,105.750000,104.459999,105.199997,105.199997,20860000
1,1977-01-11,105.199997,105.599998,103.760002,104.120003,104.120003,24100000
2,1977-01-12,104.120003,104.180000,102.750000,103.400002,103.400002,22670000
3,1977-01-13,103.400002,104.599998,103.209999,104.199997,104.199997,24780000
4,1977-01-14,104.199997,104.709999,103.370003,104.010002,104.010002,24480000
